In [ ]:
import pandas as pd
import deepxde as dde
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
H = dde.Variable(1.0)
D = dde.Variable(1.0)

In [ ]:
# '''
# 24th July, 2023 created by Likun Chen, Wuhan University
# This is for testing the PINN parameter estimation, object: syn-machine
# EXT Parameters:
#     Vi , initial terminal voltage : 1.0 pu
#     Ta , voltage regulator time constant : 0.4 sec
#     Kf , rate feedback gain : 0.03
# SYN Parameters:
#     H , inertia constant : 1.7 MWs/MVA
#     D , synchronous mechanical damping : 0.12 pu/pu
#     Xa , stator leakage reactance : 0.130 pu
#     Xd , d-axis unsaturated reactance : 1.79 pu
#     Xd' , d-axis unsaturated transient reactance : 0.169 pu
#     Xd'' , d-axis unsaturated Sub-Trans reactance : 0.135 pu
#     Xq , q-axis unsaturated reactance : 1.71 pu
#     Xq' , q-axis unsaturated transient reactance : 0.228 pu
#     Xq'' , q-axis unsaturated Sub-Trans reactance : 0.2 pu
# For 2-order equation, the state variables are rev (w) and phase angle (delta), 
# parameters waiting estimated are H , D , Eq' = Ed' = const
#     H * d2_delta/dt2 (This is dw/dt) + D * d_delta/dt + P_ex - P_mach = 0
#     P_ex is external power balance and P_mach is the mechanical power
#     time， P_ex, delta, P_mach, dw
# '''

In [ ]:
input_data = pd.read_csv('data/SYN_ON_1ST.csv')

In [ ]:
input_data = input_data[400:]

In [ ]:
input_data.Time -= 0.02

In [ ]:
input_data.set_index('Time', inplace=True)
input_data

In [ ]:
x = input_data.index.to_numpy()
x

In [ ]:
y = input_data.to_numpy()
y.shape

In [ ]:
geom = dde.geometry.TimeDomain(0, x[-1])

In [ ]:
# H * d2_delta/dt2 (This is dw/dt) + D * d_delta/dt + P_ex - P_mach = 0
def Microgrid_system(x, y):
    """Modified Lorenz system (with exogenous input).
    H * d2_delta/dt2 (This is dw/dt) + D * d_delta/dt + P_ex - P_mach = 0
    """
    time = x
    P_ex, delta, P_mach, dw = y[:, 0:1], y[:, 1:2], y[:, 2:3], y[:, 3:4]
    
    d_delta_dt = dde.grad.jacobian(y, x, i=1)
    dw_dt = dde.grad.jacobian(y, x, i=3)
    return [
        H * dw_dt + D * d_delta_dt - P_ex + P_mach
    ]


In [ ]:
# class ex_func2():
#     def __init__(self, input_data):
#         self.input_data = input_data
#     def fun(self, t):
#         t = t.reshape(-1, )
#         return self.input_data.iloc[t]['Subsystem #1|CTLs|Vars|PGEN'].to_numpy().reshape(-1, 1)

In [ ]:
observe_t = x.reshape(-1, 1)

In [ ]:
data = dde.data.PDE(
    geom,
    Microgrid_system,
    [],
    num_domain=3600,
    num_boundary=0,
    anchors=observe_t,
#     auxiliary_var_function=ex_func2(input_data).fun,
)

In [ ]:
net = dde.nn.FNN([1] + [40] * 3 + [4], "tanh", "Glorot uniform")
model = dde.Model(data, net)
model.compile("adam", lr=0.001, external_trainable_variables=[H, D])



In [ ]:
fnamevar = "variables.dat"
variable = dde.callbacks.VariableValue([H, D], period=100, filename=fnamevar)
model.train(iterations=10000, callbacks=[variable])

In [ ]:
lines = open(fnamevar, "r").readlines()
# read output data in fnamevar (this line is a long story...)
Chat = np.array(
    [
        np.fromstring(
            min(re.findall(re.escape("[") + "(.*?)" + re.escape("]"), line), key=len),
            sep=",",
        )
        for line in lines
    ]
)

In [ ]:
l, c = Chat.shape
plt.plot(range(l), Chat[:, 0], "r-")
plt.plot(range(l), Chat[:, 1], "k-")

plt.plot(range(l), np.ones(Chat[:, 0].shape) * 1.7, "r--")
plt.plot(range(l), np.ones(Chat[:, 1].shape) * 0.12, "k--")

plt.legend(["Hhat", "Dhat","True H", "True D"], loc="right")
plt.xlabel("Epoch")

In [ ]:
yhat = model.predict(observe_t)
plt.figure()
plt.plot(observe_t, y, "-", observe_t, yhat, "--")
plt.xlabel("Time")
plt.legend(["P_ex", "delta", "P_mach", "dw", "hP_ex", "hdelta", "hP_mach", "hdw"])
plt.title("Training data")
plt.show()